In [ ]:
!pip install -q --upgrade \
  gradio \
  googlesearch-python \
  beautifulsoup4 \
  requests \
  llama-cpp-python \
  transformers \
  rapidfuzz \
  spacy[transformers]



In [ ]:
from googlesearch import search
import requests
from bs4 import BeautifulSoup
from transformers import pipeline
from rapidfuzz import process
import re
import spacy


# Pre-load summarizer
summarizer_pipeline = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

# A simple list of known company names for fuzzy matching
KNOWN_COMPANIES = ["Google", "Apple", "Microsoft", "Amazon", "Meta", "Tesla", "Netflix", "Samsung", "Intel", "Nvidia", "IBM", "Adobe", "Twitter"]

# --- Agent Classes ---

class SearchAgent:
    def get_links(self, company, num_results=3):
        query = f"{company} latest news"
        return list(search(query, num_results=num_results))

class ScrapeAgent:
    def get_article_text(self, url):
        try:
            res = requests.get(url, timeout=5)
            soup = BeautifulSoup(res.text, 'html.parser')
            text = ' '.join(p.get_text() for p in soup.find_all('p'))
            return text[:3000]
        except:
            return ""

class SummarizerAgent:
    def summarize(self, text, style="formal"):
      if not text.strip():
          return "No content found to summarize."

      input_len = len(text.split())
      max_len = max(20, min(130, int(input_len * 0.6)))  # 60% of input
      min_len = max(10, int(max_len * 0.5))              # half of max_len

      try:
          summary = summarizer_pipeline(
              text,
              max_length=max_len,
              min_length=min_len,
              do_sample=False
          )[0]['summary_text']
      except Exception as e:
          return f"Error during summarization: {e}"

      if style == "casual":
          return f"Here’s the gist: {summary}"
      elif style == "bullets":
          return "• " + summary.replace('. ', '.\n• ')
      else:
          return summary

nlp = spacy.load("en_core_web_trf")  # Open-source, free

class ChatAgent:

    GENERIC_WORDS = {"news", "latest", "company", "update", "updates", "report", "headline", "business"}

    def is_company_question(self, user_input):
      keywords = ["news", "latest", "about", "update", "tell", "report", "new with"]
      return any(word in user_input.lower() for word in keywords)


    def extract_companies(self, user_input):
        doc = nlp(user_input)
        candidates = set()

        # 1. Extract ORG-type entities (like Google, Meta)
        for ent in doc.ents:
            if ent.label_ == "ORG":
                candidates.add(ent.text.strip())

        '''# 2. Also extract capitalized words longer than 3 chars (fallback)
        words = re.findall(r'\b[A-Z][a-zA-Z&\.\']+\b', user_input)
        for word in words:
            if word.lower() not in self.GENERIC_WORDS:
                candidates.add(word)'''

        # 3. Match against known companies (fuzzy matching)
        final_matches = set()
        for name in candidates:
            result = process.extractOne(name, KNOWN_COMPANIES)
            if result and result[1] > 85:
                final_matches.add(result[0])
            else:
                final_matches.add(name)  # Accept unknown name as potential company

        return list(final_matches)







/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# Initialize agents
search_agent = SearchAgent()
scrape_agent = ScrapeAgent()
summarizer_agent = SummarizerAgent()
chat_agent = ChatAgent()

'''def handle_chat(user_input, style="formal"):
    greetings = ["hi", "hello", "hey", "how are you"]

    if user_input.lower().strip() in greetings:
        return "Hi! 👋 I'm your news buddy. Ask me about any company!"

    if not chat_agent.is_company_question(user_input):
        return "I'm a bot to get only company latest news. Sorry, I could not answer these questions."

    companies = chat_agent.extract_companies(user_input)

    if not companies:
        return "Sorry, I couldn't recognize any valid company in your message. Try again with correct names."

    full_response = ""
    for company in companies:
        links = search_agent.get_links(company)
        if not links:
            full_response += f"🔹 **{company}**\nNo news links found.\n\n"
            continue

        summaries = []
        bad_links = []
        for link in links:
            article = scrape_agent.get_article_text(link)
            summary = summarizer_agent.summarize(article, style)
            if "No content" in summary or "Error during summarization" in summary:
                bad_links.append(link)
            else:
                #summaries.append(f"• {summary}\n🔗 {link}")
                summaries.append(f"• {summary}\n[Read full article]({link})")


        if summaries:
            full_response += f"🔹 **{company}**\n" + "\n\n".join(summaries) + "\n\n"
        if bad_links:
            #full_response += f"\n📎 Additional links for **{company}**:\n" + "\n".join(f"🔗 {link}" for link in bad_links) + "\n\n"
            full_response += f"\n📎 Additional links for **{company}**:\n" + "\n".join(f"[Open]({link})" for link in bad_links)


    return full_response.strip() or "No content could be fetched at the moment."'''

def handle_chat(user_input, style="formal"):
    greetings = ["hi", "hello", "hey", "how are you"]
    if user_input.lower().strip() in greetings:
        return "Hi! 👋 I'm your news buddy. Ask me about any company!"

    if not chat_agent.is_company_question(user_input):
        return "I'm a bot to get only company latest news. Sorry, I could not answer these questions."

    companies = chat_agent.extract_companies(user_input)
    if not companies:
        return "Sorry, I couldn't recognize any valid company in your message. Try again with correct names."

    full_response = ""
    for company in companies:
        links = search_agent.get_links(company)
        if not links:
            full_response += f"🔹 **{company}**\nNo news links found.\n\n"
            continue

        summaries = []
        additional_links = []
        for link in links:
            article = scrape_agent.get_article_text(link)
            summary = summarizer_agent.summarize(article, style)

            # If summary is valid, add it under the company section
            if "No content" in summary or "Error during summarization" in summary:
                additional_links.append(link)
            else:
                summaries.append(f"• {summary}\n[Read full article]({link})")

        # Add the summaries (if any)
        if summaries:
            full_response += f"🔹 **{company}**\n" + "\n\n".join(summaries) + "\n\n"

        # Add "Additional links" section (if any)
        if additional_links:
            full_response += f"📎 **Additional links for {company}:**\n" + \
                            "\n".join(f"[Open]({link})" for link in additional_links) + "\n\n"

    full_response = f"🧪 **[Debug] Companies Extracted:** {', '.join(companies)}\n\n" + full_response

    return full_response.strip() or "No content could be fetched at the moment."




In [ ]:
import gradio as gr

style_choices = ["formal", "casual", "bullets"]

gr.Interface(
    fn=handle_chat,
    inputs=[
        gr.Textbox(label="Ask about a company or just chat"),
        gr.Dropdown(choices=style_choices, label="Output Style", value="formal")
    ],
    outputs="text",
    title="📰 Smart Company News Bot",
    description="Ask about companies like Google, Meta, Apple. Get real-time news summaries in your selected style!"
).launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c8c52e2905dbd4f586.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Your max_length is set to 20, but your input_length is only 10. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=5)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://c8c52e2905dbd4f586.gradio.live
